<h1>Comparsion of different models of keyword extraction</h1>
<h3>Models:</h3>
1. Tf-idf keyword extraction: extract words with hight tf-idf score as keyword<br>
2. RAKE: (Rapid Automatic Keyword Extraction) is a well-known keyword extraction method which uses a list of stopwords and phrase delimiters to detect the most relevant words or phrases in a piece of text<br>
3. TextRank: graph-based approach of keyword extraction<br>
4. Bert<br>

For models comparsion I use datset of news articles with keywords from [here](https://www.kaggle.com/harishcscode/all-news-articles-from-home-page-media-house). For each model I count precision, recall, f1 

In [1]:
import RAKE
import pandas as pd
import nltk

from KeyWordExtractorChecking import compare_models
from StopListGenerator import generate_stop_list

In [2]:
def to_list(tags_s):
    tags = [t for t in tags_s.replace('\'', '').replace('[', '').replace(']', '').split(', ')]
    return tags

article_df = pd.read_csv('./data/articles.csv')
keywords = article_df['keywords'].apply(to_list).array
articles = article_df['articles'].array

print("Data Contains %d articles"% len(articles))
print("Mean article size: %d"% (sum(map(len, articles)) / len(articles)))
print("Mean keywords amount : %d"%(sum( map(len, keywords) ) / len(keywords)))

Data Contains 939 articles
Mean article size: 5479
Mean keywords amount : 13


In [3]:
from nlp_case.server.app.models.preprocessor.StandartPreprogressor import preprocess_text,tokenizer_regex
from nlp_case.server.app.models.text_keywords_models.Tfidf_model import Tfidf_model

In [4]:
from rake_nltk import Rake

class RAKE_model:
    def __init__(self, stopwords, max_length, name="RAKE"):
        self.stopwords = stopwords
        self.name = name
        self.model = Rake(
            punctuations=",.\"\'()[]:; ",
            stopwords=self.stopwords,
            max_length=max_length
        )
    def predict_tags(self,text):
        clean_text = ' '.join(tokenizer_regex(text, min_token_size=1))
        clean_text = preprocess_text(clean_text)
        self.model.extract_keywords_from_text(clean_text)
        keywords = self.model.get_ranked_phrases()
        return keywords


In [7]:
from summa import keywords as summa_keywords

class TextRank_model:
    def __init__(self, name="TextRank"):
        self.name = name
        self.stoplist = set(nltk.corpus.stopwords.words('english'))
        

    def predict_tags(self, text):
        clean_text = ' '.join(tokenizer_regex(text, min_token_size=1))
        clean_text = preprocess_text(clean_text, stopwords = self.stoplist)
        keywords = summa_keywords.keywords(clean_text).split()
        
        return keywords
    

In [ ]:
class My_Tfidf_model:
    def __init__(self, name="Tfidf_model"):
        self.model = Tfidf_model()
        self.name = name
        
    def predict_tags(self, text):
        keywords = self.modle.predict_keywords(text)
        
        return keywords

In [9]:
rake1 = RAKE_model(RAKE.FoxStopList(), 3, "RAKE_Model FoxStopWords") 
text_rank = TextRank_model()
tf_model = My_Tfidf_model()

In [10]:
compare_models([rake1, text_rank, tf_model],articles, keywords)

----------
RAKE_Model FoxStopWords Report
True keywords: Total 12661 Mean 13.483493
Extracted keywords: Total 175538 Mean 186.941427
Correct keywords: Total 4891 Mean 5.208733
Recall: 0.386304 Precision: 0.027863
F1: 0.025988
----------
----------
TextRank Report
True keywords: Total 12661 Mean 13.483493
Extracted keywords: Total 55867 Mean 59.496273
Correct keywords: Total 6628 Mean 7.058573
Recall: 0.523497 Precision: 0.118639
F1: 0.096720
----------


AttributeError: 'Tfidf_model' object has no attribute 'predict_tags'